Source: https://github.com/oschow/take-a-hike/blob/master/AllTrails/scrape_clean/scrape_ratings.py

In [1]:
# installs
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 5.9MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [49.8 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launc

In [2]:
# imports
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, SoupStrainer
from pymongo import MongoClient
import time
import sys
import re
import urllib
import csv

In [3]:
# Set up chromedriver
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [14]:
# Definition for logging in
def login(browser):
    browser.get('https://www.alltrails.com/login?ref=header')
    username = browser.find_element_by_name("userEmail")
    password = browser.find_element_by_name("userPassword")
    username.send_keys("221ornithomimus@gmail.com")
    password.send_keys("Dj3!120423u_9jkd_5!k34")
    browser.find_element_by_xpath("//input[@data-test-id='formButton-submit']").click()


# Scroll through the page and load all of the hikes
def get_hikes(browser):
    browser.get('https://www.alltrails.com/canada/british-columbia')
    i = 0
    while True:
        try:
            element = WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR,"[title^= 'Show more trails']" )))
            element.click()
            time.sleep(7)
            print("more trails on the way!!",str(i))
            i += 1
        except:
            break
    soup = BeautifulSoup(browser.page_source)
    return soup

# def get_photos(hike, browser):
#     browser.get(hike[0])
#     soup2 = BeautifulSoup(browser.page_source)
#     hikes = soup2.find_all(class_="photo-item gallery sm")[:3] # get first 3 photos
#     print(hikes)
#     i = 0
#     for element in hikes:
#         hike_url = element.get('href') # get url from hike
#         urllib.request.urlretrieve(hike_url, filename="/content/hike_photos/" + hike[1] + "-" + str(i) + ".jpg")
#         i += 1
#         time.sleep(5) # wait 5 seconds to avoid being blocked

# Get links to a bunch of photos
def get_photo_links(hike, browser):
    browser.get(hike[0])
    soup2 = BeautifulSoup(browser.page_source)
    hikes = soup2.find_all(class_="trail-photo lazyloaded")[:3] # get first 3 photos
    i = 0
    links = []
    for element in hikes:
        hike_url = element.get('src') # get url from hike
        links += [(hike_url, hike[1] + "-" + str(i))]
        i += 1
        time.sleep(1) # wait 1 second to avoid being blocked
    return links

In [6]:
# Log in to AllTrails
login(wd)

In [16]:
# Scroll through and load all available hikes
soup = get_hikes(wd)

hikes = soup.find_all(itemid=re.compile('/trail/\S*'))

more trails on the way!! 0
more trails on the way!! 1
more trails on the way!! 2
more trails on the way!! 3
more trails on the way!! 4
more trails on the way!! 5
more trails on the way!! 6
more trails on the way!! 7
more trails on the way!! 8
more trails on the way!! 9
more trails on the way!! 10
more trails on the way!! 11
more trails on the way!! 12
more trails on the way!! 13
more trails on the way!! 14
more trails on the way!! 15
more trails on the way!! 16
more trails on the way!! 17
more trails on the way!! 18
more trails on the way!! 19
more trails on the way!! 20
more trails on the way!! 21
more trails on the way!! 22
more trails on the way!! 23
more trails on the way!! 24
more trails on the way!! 25
more trails on the way!! 26
more trails on the way!! 27
more trails on the way!! 28
more trails on the way!! 29
more trails on the way!! 30
more trails on the way!! 31
more trails on the way!! 32
more trails on the way!! 33
more trails on the way!! 34
more trails on the way!! 35
mo

In [17]:
# Get a list of hike urls and their id numbers
hike_links = []

for hike in hikes:
    tag = hike.get('itemid').split('/')[-1]
    hike_links += [['https://www.alltrails.com' + hike.get('itemid') + '/photos', tag]]

In [18]:
# Save hike links to a csv file for backup
with open('hike_links.csv', mode='w') as f:
    writer = csv.writer(f)
    writer.writerows(hike_links)

In [19]:
photo_links = []
for link in hike_links:
    photo_links += get_photo_links(link, wd)

In [ ]:
photo_links

In [20]:
# Save photo links to a csv file
with open('photo_links.csv', mode='w') as f:
    writer = csv.writer(f)
    writer.writerows(photo_links)

VVV _____ Here be dragons _____ VVV

In [ ]:
# Download each image in the list
for hike in hike_links:
    get_photos(hike,wd)

In [ ]:
wd.get(hike_links[0][0])
soup2 = BeautifulSoup(wd.page_source)
# hikes = soup2.find_all(class='photo-item gallery sm')

In [ ]:
photos = soup2.find_all(class_="photo-item gallery sm")[:5]

In [ ]:
photos

In [ ]:
# Go to page
wd.get("https://www.alltrails.com/canada/british-columbia")

# For each entry, get itemid
soup = BeautifulSoup(wd.page_source)

hikes = soup.select('div data-hydrate')

In [ ]:
hikes = soup.select('div data-hydrate')

In [ ]:
hikes

[]